In [1]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

INFO - Logger configured


In [2]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [3]:
from src.DataLoader import room_list_generator,room_r_tree_generator
room_list=room_list_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_list)

In [4]:
from src.DataLoader import build_receiver_set
receiver_set= build_receiver_set(receiver_gdf,room_list,room_r_tree)
receiver_set

{'A8032A31204E': Receiver(room=Room(uid=21884, name='4A443', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>), point=<POINT (2.201 48.713)>),
 'C45BBE39F42A': Receiver(room=Room(uid=21283, name='', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>), point=<POINT (2.201 48.713)>),
 'C45BBE39F9AE': Receiver(room=Room(uid=21849, name='4A467', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>), point=<POINT (2.201 48.713)>),
 'A8032A311FAA': Receiver(room=Room(uid=21867, name='4A441', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>), point=<POINT (2.201 48.713)>),
 'C45BBE39F56A': Receiver(room=Room(uid=21283, name='', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>), point=<POINT (2.201 48.713)>),
 'C45BBE37B346': Receiver(room=Room(uid=21885, name='4A449', polygon=<POLYGON ((2.201 48.713, 2

In [5]:
Threshold1=-45
Threshold2=-70

In [6]:
from src.DataLoader import Receiver
from shapely.geometry import Point,Polygon

#need in the class: receiver_set
def zone_selection(receiver:Receiver,rssi:int)->Polygon:
    """
    Return the predicted zone for the selected receiver receiving the rssi 
    
    Parameters
    ---
    Input:
    receiver: Receiver
    rssi: int
    
    Output:
    Polygon
    """
    if rssi>Threshold1:
        return receiver.room.polygon
    

In [7]:
def test_case1(receiver_set):
    receiver=receiver_set['A8032A31204E']
    rssi=-40
    zone=zone_selection(receiver,rssi)
    assert zone==receiver.room.polygon,f'Error: {zone_selection(receiver,rssi)} should be {receiver.room.polygon}'
test_case1(receiver_set)

In [8]:
receiver=receiver_set['A8032A31204E']
receiver.room


Room(uid=21884, name='4A443', polygon=<POLYGON ((2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48.713, 2.201 48....>)